In [1]:
import tensorflow as tf
import pprint
import numpy as np
import pandas as pd
import hopsworks
import warnings
warnings.filterwarnings('ignore')


2025-04-03 00:05:45.661000: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-03 00:05:45.757808: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-03 00:05:46.515409: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-03 00:05:46.516340: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-03 00:05:46.643689: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:

# Connect to Hopsworks Feature Store
project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()


2025-04-03 00:06:06,920 INFO: Initializing external client
2025-04-03 00:06:06,921 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-03 00:06:08,703 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788


In [7]:

# Load candidate model from Model Registry
model = mr.get_model(
    name="candidate_model",
    version=1,
)
model_path = model.download()

In [8]:
candidate_model = tf.saved_model.load(model_path)


In [5]:

# Retrieve feature view
feature_view = fs.get_feature_view(
    name="event_retrieval_2",
    version=1,
)


In [6]:

# Load training data
train_df, val_df, test_df, _, _, _ = feature_view.train_validation_test_split(
    validation_size=0.1, 
    test_size=0.01,
    description='Event retrieval dataset splits',
)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.18s) 


In [9]:
train_df.head(3)

,interaction_id,user_id,user_city,age,user_interests,event_id,event_type,event_city
0,4140ef03-afce-4fe5-a9a9-8e43fd45dfaa,LT819S,Sydney,26,sports literature cinema,OI841N,Food & Drink,Tokyo
1,BI623H,UP287J,Mumbai,56,tech food travel,IO571I,Technology,Mumbai
2,08e342ae-f9b5-4358-9f41-cf89671b9ae2,RG723K,Mumbai,32,literature music,YW813D,Education & Learning,Paris


In [10]:
# Get the list of input features for the candidate model from the model schema
model_schema = model.model_schema['input_schema']['columnar_schema']
candidate_features = [feat['name'] for feat in model_schema]

In [12]:
# Select the candidate features from the training DataFrame
item_df = train_df[candidate_features]

# Drop duplicate rows based on the 'video_id' column to get unique candidate items
item_df.drop_duplicates(subset="event_id", inplace=True)

item_df.head(3)

,event_id,event_type,event_city
0,OI841N,Food & Drink,Tokyo
1,IO571I,Technology,Mumbai
2,YW813D,Education & Learning,Paris


In [15]:
# # Create a TensorFlow dataset from the item DataFrame
# item_ds = tf.data.Dataset.from_tensor_slices(
#     {col: item_df[col] for col in item_df})

# # Compute embeddings for all candidate items using the candidate_model
# candidate_embeddings = item_ds.batch(128).map(
#     lambda x: (x["event_id"], candidate_model(x))
# )

In [16]:
# # Concatenate all article IDs and embeddings from the candidate_embeddings dataset
# all_article_ids = tf.concat([batch[0] for batch in candidate_embeddings], axis=0)
# all_embeddings = tf.concat([batch[1] for batch in candidate_embeddings], axis=0)

# # Convert tensors to numpy arrays
# all_article_ids_np = all_article_ids.numpy()
# all_embeddings_np = all_embeddings.numpy()

# # Convert numpy arrays to lists
# items_ids_list = all_article_ids_np.tolist()
# embeddings_list = all_embeddings_np.tolist()

# # Create a DataFrame
# data_emb = pd.DataFrame({
#     'video_id': items_ids_list, 
#     'embeddings': embeddings_list,
# })
# data_emb['video_id'] = data_emb['video_id'].str.decode('utf-8')

# data_emb.head()

,video_id,embeddings
0,OI841N,"[3.4449751377105713, -5.7353291511535645, -0.6..."
1,IO571I,"[4.088526725769043, -5.447272777557373, -0.793..."
2,YW813D,"[3.44762921333313, -5.54926061630249, 0.143870..."
3,RM794Y,"[3.4445602893829346, -5.764412879943848, -0.80..."
4,UK320K,"[3.249997138977051, -8.640835762023926, 0.9318..."


In [13]:

# Extract unique event IDs
event_ids = train_df["event_id"].unique().tolist()
print(f"Number of unique events: {len(event_ids)}")

# Create a dataframe with only the candidate features
CANDIDATE_FEATURES = ["event_id", "event_type", "event_city"]
event_df = train_df[CANDIDATE_FEATURES]
event_df.drop_duplicates(subset="event_id", inplace=True)

# Compute embeddings for all events
embeddings = []
for _, row in event_df.iterrows():
    # Prepare input for the candidate model
    input_data = {
        "event_id": tf.constant([row["event_id"]]),
        "event_type": tf.constant([row["event_type"]]),
        "event_city": tf.constant([row["event_city"]])
    }
    # Generate embedding
    embedding = candidate_model(input_data).numpy().tolist()[0]
    embeddings.append(embedding)

# Create dataframe with event IDs and their embeddings
embeddings_df = pd.DataFrame({
    "event_id": event_df["event_id"],
    "embeddings": embeddings
})


Number of unique events: 15402


In [14]:
embeddings_df.head()

,event_id,embeddings
0,OI841N,"[3.4449751377105713, -5.7353291511535645, -0.6..."
1,IO571I,"[4.088526725769043, -5.447272777557373, -0.793..."
2,YW813D,"[3.44762921333313, -5.54926061630249, 0.143870..."
3,RM794Y,"[3.4445602893829346, -5.764412879943848, -0.80..."
4,UK320K,"[3.249997138977051, -8.640835762023926, 0.9318..."


## <span style="color:#ff5f27">🪄 Feature Group Creation </span>

Now you are ready to create a feature group for your candidate embeddings.

To begin with, you need to create your Embedding Index where you will specify the name of the embeddings feature and the embeddings length.
Then you attach this index to the FG.

In [21]:
from hsfs import embedding

# Create the Embedding Index
emb = embedding.EmbeddingIndex()

emb.add_embedding(
    "embeddings",                           # Embeddings feature name
    len(embeddings_df["embeddings"].iloc[0]),    # Embeddings length
)

In [22]:
# Get or create the 'candidate_embeddings_fg' feature group
candidate_embeddings_fg = fs.get_or_create_feature_group(
    name="events_candidate_embeddings_fg",
    embedding_index=emb,                    # Specify the Embedding Index
    primary_key=['event_id'],
    version=1,
    description='Embeddings for each events',
    online_enabled=True,
)

candidate_embeddings_fg.insert(embeddings_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fg/1423087


Uploading Dataframe: 100.00% |██████████| Rows 15402/15402 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: events_candidate_embeddings_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1220788/jobs/named/events_candidate_embeddings_fg_1_offline_fg_materialization/executions


(Job('events_candidate_embeddings_fg_1_offline_fg_materialization', 'SPARK'),
 None)

In [24]:

# Get or create the 'candidate_embeddings' feature view
feature_view = fs.get_or_create_feature_view(
    name="events_candidate_embeddings",
    version=1,
    description='Embeddings of each event',
    query=candidate_embeddings_fg.select(["event_id"]),
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fv/events_candidate_embeddings/version/1


In [ ]:

# Create a feature view joining event metadata with embeddings
event_embeddings_fv = fs.get_or_create_feature_view(
    name="event_embeddings",
    version=1,
    query=candidate_embeddings_fg.select_all().join(
        fs.get_feature_group("events", version=1).select(
            ["event_id", "title", "event_type", "event_city", "duration", "event_indoor_capability"]
        ),
        on=["event_id"]
    )
)

print("✅ Embeddings created and stored successfully!")
